# Forward and backward pass

In [1]:
import pickle, gzip, math, os, time, shutil, torch
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from torch import tensor
from fastcore.test import test_close

In [2]:
torch.manual_seed(42)
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)
plt.style.use('fast')

In [3]:
path_data = Path('./data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: 
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [4]:
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

# Foundations
## Basic architecture

In [5]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

In [6]:
num_hidden = 50

In [7]:
w1 = torch.randn(m, num_hidden)
b1 = torch.zeros(num_hidden)

w2 = torch.randn(num_hidden, 1)
b2 = torch.zeros(1)

In [8]:
def lin(x, w, b): return x@w + b

In [9]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [10]:
def relu(x): return x.clamp_min(0.)

In [11]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [12]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [13]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

## Loss: MSE

For now, technically we should be using cross-entropy

In [14]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [15]:
(res.squeeze() - y_valid).shape

torch.Size([10000])

In [16]:
res[:, 0].shape # Equivalent to the above squeeze

torch.Size([10000])

In [17]:
y_train, y_valid = y_train.float(), y_valid.float()
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [18]:
def mse(output, target): return ((output[:, 0] - target)**2).mean()

In [19]:
mse(preds, y_train)

tensor(4308.76)

# Gradients and backward pass

In [20]:
!pip install sympy

In [21]:
from sympy import symbols, diff

In [22]:
x, y = symbols('x y')
diff(x**2, x), diff(x**2, y)

(2*x, 0)

In [23]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t() # the derivative is just the weights, multiply by the output gradients bc of chain rule
    print(f'inp.g {inp.g}')
    w.g = inp.t() @ out.g
    print(f'w.g {w.g}')
    b.g = out.g.sum(0)
    print(f'b.g {b.g}')

In [24]:
def forward_and_backward(inp, target):
    # forward
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:, 0] - target
    loss = diff.pow(2).mean()
    print(loss)
    
    # backward
    out.g = 2.*diff[:,None] / inp.shape[0] # divide by batchsize as before we took the mean
    print(f"mse.g {out.g}")
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g # slope is 1 if > 0, otherwise 0
    lin_grad(inp, l1, w1, b1)
    

In [25]:
forward_and_backward(x_train, y_train)

tensor(4308.76)
mse.g tensor([[-0.00],
        [-0.00],
        [ 0.00],
        ...,
        [-0.00],
        [-0.00],
        [-0.00]])
inp.g tensor([[     0.00,     -0.00,     -0.00,  ...,      0.00,     -0.00,      0.00],
        [     0.00,     -0.00,     -0.00,  ...,      0.00,     -0.01,      0.01],
        [    -0.00,      0.00,      0.00,  ...,     -0.00,      0.00,     -0.00],
        ...,
        [     0.00,     -0.00,     -0.00,  ...,      0.00,     -0.00,      0.00],
        [     0.00,     -0.00,     -0.00,  ...,      0.00,     -0.00,      0.00],
        [     0.00,     -0.00,     -0.00,  ...,      0.00,     -0.00,      0.00]])
w.g tensor([[ -869.79],
        [ -622.49],
        [ -165.30],
        [ -468.34],
        [ -837.93],
        [ -687.30],
        [ -559.73],
        [ -637.38],
        [ -176.24],
        [  -10.81],
        [ -993.01],
        [ -533.38],
        [ -730.86],
        [  -54.01],
        [ -451.91],
        [ -156.28],
        [ -567.42],
      

In [26]:
# Save for later testing
def get_grad(x): return x.g.clone()
checks = w1, w2, b1, b2, x_train
grads = w1g, w2g, b1g, b2g, ig = tuple(map(get_grad, checks))

In [27]:
# cheat using pytorch
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12, w22, b12, b22, xt2 = tuple(map(mkgrad, checks))

In [28]:
def forward(inp, target):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, target)

In [29]:
loss = forward(xt2, y_train)
loss.backward()

In [30]:
from IPython.core.debugger import set_trace

In [31]:
for i, (a, b) in enumerate(zip(grads, ptgrads)): 
    test_close(a, b.grad, eps=1e-3)

# Refactor

In [32]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [33]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0) 

In [34]:
class Mse():
    def __call__(self, inp, target):
        self.inp, self.target = inp, target
        self.out = mse(inp, target)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.target).unsqueeze(-1) / self.target.shape[0]

In [35]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, target):
        for l in self.layers: x = l(x)
        return self.loss(x, target)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [36]:
model = Model(w1, b1, w2, b2)

In [37]:
loss = model(x_train, y_train); loss

tensor(4308.76)

In [38]:
model.backward()

In [39]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

# Module.forward()

In [40]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception("not implmented")
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception("not implemented")

In [41]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp > 0).float() * out.g

In [42]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
    def forward(self, inp): return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)         

In [43]:
class Mse(Module):
    def forward(self, inp, target): 
        self.diff = inp.squeeze() - target
        return self.diff.pow(2).mean()
    def bwd(self, out, inp, target): 
        inp.g = 2. * self.diff.unsqueeze(-1) / target.shape[0]        

In [44]:
model = Model(w1, b1, w2, b2)

In [45]:
loss = model(x_train, y_train)

In [46]:
model.backward()

In [47]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

# Autograd

In [48]:
from torch import nn
import torch.nn.functional as F

In [49]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
        
    def forward(self, inp): 
        result = inp@self.w + self.b
        return result

In [50]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in, nh), nn.ReLU(), Linear(nh, n_out)]
        
    def __call__(self, x, target):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, target[:, None])
        

In [51]:
model = Model(m, num_hidden, 1)
loss = model(x_train, y_train)
loss.backward()


In [52]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])